In [ ]:
from hofss import Factor

factors = Factor.parse_from_file("/home/boonstra/git_repos/hofss/data/hofs_frequencies_and_multipliers.csv")

In [ ]:
from matplotlib.axes._axes import Axes
import matplotlib.pyplot as plt
import numpy as np

def plot_factor_pdf(factor: Factor, ax: Axes, effect: str = "neg") -> Axes:

    effect = effect.lower()
    if effect not in ["pos", "neg"]:
        raise ValueError("effect must be either one of: 'pos', 'neg'")
    
    cdf_probabilities = [0, 0.05, 0.5, 0.95, 1.0]
    multipliers = []
    for percentile in ["lower", "5", "50", "95", "upper"]:
        attribute_name = f"m_{effect}_{percentile}"
        multipliers.append(getattr(factor, attribute_name))

    plot_points = [(multipliers[0],0)]
    sum_pdf = 0
    for i in range(4):
        probability = (cdf_probabilities[i+1]-cdf_probabilities[i]) / (multipliers[i+1] - multipliers[i])
        plot_points.append((multipliers[i], probability))
        plot_points.append((multipliers[i+1], probability))
        sum_pdf += probability * (multipliers[i+1] - multipliers[i])

    plot_points.append((multipliers[-1], 0))
    ax.plot(*zip(*plot_points), label=factor.description)
    return ax



In [ ]:
fig, axs = plt.subplots(2)
scale_factor = 3
fig.set_size_inches(w=scale_factor*80/25.4, h=scale_factor*60/25.4)
for ax in axs:
    ax.set_ylabel("PDF $f(x)$")

axs[0].set_title("Positive effect")
axs[1].set_xlabel("Multiplier value [-]")
axs[1].set_title("Negative effect")


for factor_id in ["F1", "F3", "F2", "F4"]:
    factor = list(filter(lambda x: x.name == factor_id, factors))[0]
    axs[0] = plot_factor_pdf(factor, axs[0], "pos")
    axs[1] = plot_factor_pdf(factor, axs[1], "neg")


# ax[0].set_xscale("log")
axs[1].set_xscale("log")

axs[1].legend(loc='upper center', bbox_to_anchor=(0.5,-0.2), ncols=4)

plt.tight_layout()
plt.show()

In [ ]:
fig.savefig("PDF_hofs.png")